#  ETL ( Extract, Transform, Load)
ETL generically refers to the process of taking some data from source systems, (such as your CRM, ERP, or Marketing stack) transforming said data, and then loading it to some end system / application. 

**Extract**

Taking data from a source system
- Building pipelines to pull data from some source system(s) 
- Considerations around volume, speed (performance), cost

**Transform**

Manipulating Data for some end purpose, common transformations can include:
- Aggregation 
- Applying logic or calculations
- Joins, sorts, or pivots
- Applying data validation

**Load**

Loading or sending the data to some end system or application
- Output to a data warehouse
- Generating  flat files
- Updating / Creating tables in a database 

ETL is typically the domain of a data engineer, since there are typically considerations around volume, speed, and cost of shuttling data around a large enterprise systems. However, today you are typically seeing the rise of various developer positions that typically deal with creating automation around smaller volumes of data / non-critical systems.


![ETL](https://www.altexsoft.com/media/2019/06/word-image-29.png)

# Before Connecting to the DB
### Let's talk a little a bit about credentials in scripts

Items to be aware of with data connections in scripts:

- Do not store plain text passwords in scripts!
- Preferablly there should be some sort of encryption on credentials as you pass them
- Its generally a good practice to keep credentialing values in a seperate file to be referenced by your scripts


### How do we do this?

Network security, encryption, and best security practices are a entire domain within computer science. We will not be able to cover every aspect of it that you may need to know to optimally protect your systems or connections.

What I'll attempt to do here is show you a few techniques that you can use to make yourself more defensible and keep your credential information from being stored directly in your scripts.

# Building a JSON to push and pull credential information

In [31]:
#Creating some token to reuse in scripts so you don't need to store credentials / sensitive info in the script
import json 

#example token created with a dictionary
example_token = {'key1':'value1','key2':'value2','key3':'value3'}

#saving that token(dictionary) as a json file
example = json.dumps(example_token)
f = open("example_token.json","w")
f.write(example)
f.close()

In [32]:
#loading a json token back into memory
with open('example_token.json', "r") as file:
    token = json.load(file)

#call the value of a particular key
print(token['key1'])

value1


# Lets Build our Credential

What do we need in order to establish a connection to a MS based DB?

- Host name (Name of the machine the DB is on)
- User name 
- Password (***NOT*** *IN PLAIN TEXT*)
- Database

So how do we get our password into the credential without adding it in plain text?

ENTER - Cryptography!

In [34]:
from cryptography.fernet import Fernet

#generate a key that will allow you to encrypt / decrypt your password
key = Fernet.generate_key()

print(f'Your key is: {key}') # we'll store this in a seperate key file to decrypt our password

#create a cipher suite that will allow for encryption

#prefix string with b to pass it as bytecode
pword = b'supersecret_password' 

cipher_suite = Fernet(key) # used to encrypt / decrypt the password

ciphered_text = cipher_suite.encrypt(pword) #we'll store this in our credential file
print('')
print(f'Your encrypted password: {ciphered_text}')


#decrypt password 
unciphered_text = cipher_suite.decrypt(ciphered_text)
print('')
print(f'You password after decription: {unciphered_text}')

Your key is: b'sqbh4JjXzMeUFF0l_oPEXe2VEser-RbjSPLl0A26b_0='

Your encrypted password: b'gAAAAABe0C3f3UlPxYB2UWclMW5fyWDNC5os3RL50wbcMtxqIf4I9TzXYaGQqjS21fZDPHK7Ya44B0fxTa1_n-PQhKn9nd82SZmr3AgKtf0liv-ATPxiEdY='

You password after decription: b'supersecret_password'


In [35]:
#lets create our credential to connect

#example token created with a dictionary
sql_token = {'server':r'server_name',
             'user':r'username',
             'password':r'THIS IS NOT THE PASSWORD', # put the ciphered password here
             'database':r'db_name'}

#saving that token(dictionary) as a json file
sql = json.dumps(sql_token)
f = open("sql_token.json","w")
f.write(sql)
f.close()

#do the same thing to create a key file!

# SQL ALCHEMY / PYMSSQL



In [36]:
#loading a json token back into memory
with open('sql_token_real.json', "r") as file:
    token = json.load(file)
    
with open('sql_key_real.json','r') as file:
    key = json.load(file)

In [37]:
import pymssql
import pandas as pd

In [42]:
cipher_suite = Fernet(str.encode(key['key']))


conn = pymssql.connect(host = token['server'], 
                       user = token['user'],
                       password = cipher_suite.decrypt(str.encode(token['password'])).decode(),
                       database = token['database'])

stmt = r"""SELECT TOP(100) * FROM DBTABLE"""

df=pd.read_sql_query(stmt,conn)

In [43]:
df.head()

,LogId,Date,NetworkIdentity,MachineName,UserFullName,Title,Office,Studio,Region,ProjectManager,...,ElapsedTime,CentralModelGuid,ProjectGuid,ModelGuid,Compacted,Comments,TimedEvent,LocalTime,LocalDate,FileCreatedDate
0,14529875,2019-12-01 00:11:11.717,18241,SFCOLOVDIHF0045,None,None,None,None,None,,...,57.2946338,,,,False,,Sync,None,None,Nov 30 2019 11:01AM
1,14529876,2019-12-01 00:16:23.893,28215,LA-PC0PDTCT,None,None,None,None,None,,...,44.6395553,,,,False,,Sync,None,None,Nov 30 2019 1:18PM
2,14529877,2019-12-01 00:20:19.853,28215,LA-PC0PDTCT,None,None,None,None,None,,...,46.4639718,,,,False,,Sync,None,None,Nov 30 2019 1:18PM
3,14529878,2019-12-01 00:21:05.007,28375,SFCOLOVDIHB0001,None,None,None,None,None,,...,179.2089859,,,,,,Open,None,None,Nov 30 2019 5:15PM
4,14529882,2019-12-01 00:31:26.240,27500,ATCOLOVDIHE0096,None,None,None,None,None,,...,8.9010176,,bd50312c-1a29-4cf0-83ca-887cc2b45efd,b733f499-61e8-41ec-bbd2-f5c366ee09e1,False,,Sync,None,None,


In [44]:
#Output our SQL query data pull
df.to_csv('cad_logging.csv')